In [1]:
import requests
import calendar
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.select import Select
from webdriver_manager.chrome import ChromeDriverManager

import pandas as pd
import os
import re
import time
import numpy as np

In [2]:
#####       example       #####

email = 'xianlin.ding@sciencespo.fr'
password = '099115Keep12??'
path = os.getcwd()
path = re.sub('Code','Result',path)

In [3]:
# go into the Eruopresse website
url = "https://acces-distant.sciencespo.fr/fork?https://nouveau.europresse.com/access/ip/default.aspx?un=politique2T_1"
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get(url)
driver.find_element(By.ID, 'username').send_keys(email)
driver.find_element(By.ID, 'password').send_keys(password)
time.sleep(3)
driver.find_element(By.CLASS_NAME, 'btn-submit').click()

<ipython-input-3-4fdf21dd8406>:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [4]:
# choose the criteria of searching
driver.find_element(By.CLASS_NAME, 'lnk-text').click()
time.sleep(3)
driver.find_element(By.ID, 'specific-sources-rd').click()
time.sleep(5)
driver.find_element(By.ID, 'sf_247').click() # le monde

In [5]:
# period: 1/oct/2022 - 1/dec/2022
Select(driver.find_element(By.ID, 'DateFilter_DateRange')).select_by_value('10')

Select(driver.find_elements(By.CLASS_NAME, 'year')[1]).select_by_value('2022')
Select(driver.find_elements(By.CLASS_NAME, 'year')[2]).select_by_value('2022')

Select(driver.find_elements(By.CLASS_NAME, 'month')[1]).select_by_value('10')
Select(driver.find_elements(By.CLASS_NAME, 'month')[2]).select_by_value('12')

Select(driver.find_elements(By.CLASS_NAME, 'day')[1]).select_by_value('1')
Select(driver.find_elements(By.CLASS_NAME, 'day')[2]).select_by_value('1')

In [6]:
driver.find_element(By.ID, 'btnSearch').click()

In [12]:
# get web contents
web_content = driver.page_source
obj = BeautifulSoup(web_content,'lxml').body
obj = obj.aside.div.div.div.div.div
obj = obj.find_all(name='span', attrs={"class":"resultOperations-count"})[0]
obj = int(re.sub(r'\xa0','',obj.contents[0]))
obj

In [ ]:
driver.back()

In [14]:
#####       actual scraping       #####

# define the function

def scrape_all_articles(journal_code, year_start, year_end):
    
    year = list(range(year_start,year_end+1))
    month = list(range(1,13))
    
    info = pd.DataFrame(columns = ['year', 'month', 'number'])
    i=0
    for year_i in year:
        for month_i in month:
            
            while True:
                try:
                    obj = scrape(journal_code, year_i, month_i)
                    break
                except:
                    print('Maximum access times reach, sleep 5 minutes.')
                    time.sleep(300)    
            
            info.loc[i] = [year_i, month_i, obj]
            print('Year ', year_i,', Month ', month_i, ', ', obj, ' finished.')
            i=i+1
            
            
    return info


In [15]:
def scrape(journal_code, year_i, month_i):
    
    email = 'xianlin.ding@sciencespo.fr'
    password = '099115Keep12??'
    
    # go into the Eruopresse website
    url = "https://acces-distant.sciencespo.fr/fork?https://nouveau.europresse.com/access/ip/default.aspx?un=politique2T_1"
    driver = webdriver.Chrome(ChromeDriverManager().install())
    driver.get(url)
    driver.find_element(By.ID, 'username').send_keys(email)
    driver.find_element(By.ID, 'password').send_keys(password)
    time.sleep(3)
    driver.find_element(By.CLASS_NAME, 'btn-submit').click()
    
    # choose the criteria of searching
    driver.find_element(By.CLASS_NAME, 'lnk-text').click()
    time.sleep(3)
    driver.find_element(By.ID, 'specific-sources-rd').click()
    time.sleep(5)
            
    # choose the journal
    try:
        driver.find_element(By.ID, journal_code).click()
    except:
        time.sleep(20)
        driver.find_element(By.ID, journal_code).click()
                
    # choose the searching period
    day_start = 1
    day_end = calendar.monthrange(year_i,month_i)[1]
    
    Select(driver.find_element(By.ID, 'DateFilter_DateRange')).select_by_value('10')

    Select(driver.find_elements(By.CLASS_NAME, 'year')[1]).select_by_value(str(year_i))
    Select(driver.find_elements(By.CLASS_NAME, 'year')[2]).select_by_value(str(year_i))
    
    Select(driver.find_elements(By.CLASS_NAME, 'day')[1]).select_by_value(str(day_start))
    Select(driver.find_elements(By.CLASS_NAME, 'day')[2]).select_by_value(str(day_end))
    
    Select(driver.find_elements(By.CLASS_NAME, 'month')[1]).select_by_value(str(month_i))
    Select(driver.find_elements(By.CLASS_NAME, 'month')[2]).select_by_value(str(month_i))      
            
    driver.find_element(By.ID, 'btnSearch').click()
    time.sleep(5)
    
    # get web contents
    web_content = driver.page_source
    obj = BeautifulSoup(web_content,'lxml').body
    obj = obj.aside.div.div.div.div.div
    obj = obj.find_all(name='span', attrs={"class":"resultOperations-count"})[0]
    obj = int(re.sub(r'\xa0','',obj.contents[0]))
    
    driver.close()
    
    return obj


In [16]:
# Le Monde
journal_code = 'sf_247'

In [17]:
info = scrape_all_articles(journal_code, 2013, 2022)

<ipython-input-15-9212e9685e63>:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


Year  2013 , Month  1 ,  3120  finished.
Year  2013 , Month  2 ,  3145  finished.
Year  2013 , Month  3 ,  3512  finished.
Year  2013 , Month  4 ,  3196  finished.
Year  2013 , Month  5 ,  3451  finished.
Year  2013 , Month  6 ,  3291  finished.
Year  2013 , Month  7 ,  2997  finished.
Year  2013 , Month  8 ,  2417  finished.
Year  2013 , Month  9 ,  3078  finished.
Maximum access times reach, sleep 5 minutes.
Maximum access times reach, sleep 5 minutes.
Year  2013 , Month  10 ,  3195  finished.
Year  2013 , Month  11 ,  3158  finished.
Year  2013 , Month  12 ,  2862  finished.
Year  2014 , Month  1 ,  2862  finished.
Year  2014 , Month  2 ,  2826  finished.
Maximum access times reach, sleep 5 minutes.
Year  2014 , Month  3 ,  3255  finished.
Year  2014 , Month  4 ,  3589  finished.
Year  2014 , Month  5 ,  5084  finished.
Year  2014 , Month  6 ,  4948  finished.
Year  2014 , Month  7 ,  4799  finished.
Year  2014 , Month  8 ,  4022  finished.
Year  2014 , Month  9 ,  5924  finished.
Y

[WDM] - Downloading: 100%|████████████████████████████████████████████████████████| 6.79M/6.79M [00:00<00:00, 39.1MB/s]


Year  2021 , Month  2 ,  2012  finished.
Year  2021 , Month  3 ,  2257  finished.
Year  2021 , Month  4 ,  2140  finished.
Year  2021 , Month  5 ,  2023  finished.
Year  2021 , Month  6 ,  2537  finished.
Maximum access times reach, sleep 5 minutes.
Year  2021 , Month  7 ,  2057  finished.
Year  2021 , Month  8 ,  1527  finished.
Year  2021 , Month  9 ,  2322  finished.
Year  2021 , Month  10 ,  2430  finished.
Year  2021 , Month  11 ,  2166  finished.
Year  2021 , Month  12 ,  2241  finished.
Year  2022 , Month  1 ,  2142  finished.
Year  2022 , Month  2 ,  2120  finished.
Year  2022 , Month  3 ,  2439  finished.
Year  2022 , Month  4 ,  2555  finished.
Year  2022 , Month  5 ,  2229  finished.
Year  2022 , Month  6 ,  2534  finished.
Year  2022 , Month  7 ,  1919  finished.
Year  2022 , Month  8 ,  1561  finished.
Year  2022 , Month  9 ,  2335  finished.
Year  2022 , Month  10 ,  2329  finished.
Year  2022 , Month  11 ,  2192  finished.
Year  2022 , Month  12 ,  2079  finished.


In [19]:
info.tail()

,year,month,number
115,2022,8,1561
116,2022,9,2335
117,2022,10,2329
118,2022,11,2192
119,2022,12,2079


In [20]:
path = os.getcwd()
path = re.sub('Code','Result',path)
info.to_csv(path + '\\data_raw_v2\\'+ 'Le Monde_All' + '.csv')

In [13]:
# scrape('sf_247', 2015, 2)

<ipython-input-12-9212e9685e63>:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


3218

In [44]:
calendar.monthrange(2013,2)

(4, 28)

In [34]:
generate_period(2013, 2022)

['2013',
 '2014',
 '2015',
 '2016',
 '2017',
 '2018',
 '2019',
 '2020',
 '2021',
 '2022']

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]

In [25]:
import pandas as pd
import datetime
import os

In [11]:
# path = os.getcwd()
path = os.path.abspath('..')

data = pd.read_csv(path+'\\Result\\data_raw\\Le Monde_2013_1_4.csv',
                   index_col=0)

In [29]:
# sort by date
data = data.sort_values(by='date')
data = data.reset_index(drop='True')
data.head(10)

,date,publication,words,author,title,intro
0,2013-01-02,Le Monde,786mots,Clément Lacombe et Audrey Tonnelier,2013 sous le signe de l'optimisme en Bourse,"... à appeler l'Europe à l'aide, prévient..."
1,2013-01-02,Le Monde,977mots,Isabelle Regnier,Avis de tempête sur le cinéma français,"... préachetés par France Télévisions, ce..."
2,2013-01-03,Le Monde,278mots,Jean-Pierre Tenoux,"Besançon, champion français des économies d'én...","... nom de Cit'ergie, pour distinguer les..."
3,2013-01-03,Le Monde,629mots,Michel Noblecourt,"2013, année charnière pour le syndicalisme fra...","Sur fond de climat social déprimé, où l'i..."
4,2013-01-04,Le Monde,315mots,C. Du.,Il n'y aura bientôt plus de fonctionnaires au ...,"... PDG, a été nommé à la tête du groupe ..."
5,2013-01-04,Le Monde,1514mots,Jean-Marc Ayrault,Pour un nouveau modèle français,... Notre nouveau modèle de développement...
6,2013-01-04,Le Monde,667mots,Cécile Ducourtieux,Orange s'apprête à tourner la page France Télécom,"... la manoeuvre, il y a quelques années...."
7,2013-01-04,Le Monde,68mots,NaN,Alerte rouge à La Réunion pour le cyclone Dumile,L'alerte cyclonique rouge est entrée e...
8,2013-01-05,Le Monde,735mots,Jean-Hervé Lorenzi,"En 2013, augmentons l'emploi des jeunes","Il y a les faits, le climat général, les ..."
9,2013-01-05,Le Monde,546mots,Olivier Truc,La Norvège tentée par le nucléaire pour exploi...,... renforcer la durabilité de l'énergie ...


In [27]:
list(data['date'])[0]

'2013-01-02'

In [28]:
datetime.datetime('2013-01-02')+ datetime.timedelta(7)

TypeError: an integer is required (got type str)